In [5]:
import subprocess
import matplotlib.pyplot as plt
import os
from constants import OPENAI_API_KEY, BITWIDTH
from parse_functions import get_pareto_points, get_points

Генерируем все возможные деревья

In [6]:
folders = [f'bitwidth_{BITWIDTH}', f'bitwidth_{BITWIDTH}_output']

for folder in folders:
    if os.path.exists(folder):
        subprocess.run(f"rm -rf {folder}", shell=True, check=True)

In [ ]:
subprocess.run(["python3", "0_generate_prefix_trees_code.py"], check=True)

Проверяем их

In [ ]:
subprocess.run(["python3", "1_check_input_files.py"], check=True)

Генерируем параметры

In [ ]:
subprocess.run(["python3", "2_generate_params.py"], check=True)

Запускаем синтез

In [ ]:
subprocess.run(f"export BIT_WIDTH={BITWIDTH} && yosys 3_run_synthesis.tcl", shell=True, check=True)

Получаем значения площади и задержки

In [ ]:
subprocess.run(["python3", "4_get_delay_sta.py"], check=True)

Строим графики

In [ ]:
res_dir = f'bitwidth_{BITWIDTH}_output'
df = get_points(res_dir)

pareto_front = get_pareto_points(df)

plt.scatter(df.area, df.delay, alpha=0.5)
plt.scatter(pareto_front.area, pareto_front.delay, c='r')
plt.grid()

Очищаем папки и запускаем LLM-генератор

In [ ]:
subprocess.run(f'rm -rf bitwidth_{BITWIDTH}/* bitwidth_{BITWIDTH}_output/*', shell=True, check=True)

In [ ]:
subprocess.run(["python3", "llm_generator.py"], check=True)

Проверяем структур

In [ ]:
subprocess.run(["python3", "1_check_input_files.py"], check=True)

Запускаем синтез

In [ ]:
subprocess.run(f"export BIT_WIDTH={BITWIDTH} && yosys 3_run_synthesis.tcl", shell=True, check=True)

Получаем площадь и задержку

In [ ]:
subprocess.run(["python3", "4_get_delay_sta.py"], check=True)

Строим графики (красные точки - Парето-Фронт для LLM-сумматора, зеленые - для прошлых)

In [ ]:
res_dir = f'bitwidth_{BITWIDTH}_output'
df_llm = get_points(res_dir)

pareto_front_llm = get_pareto_points(df_llm)

plt.scatter(df_llm.area, df_llm.delay, alpha=0.5)
plt.scatter(pareto_front_llm.area, pareto_front_llm.delay, c='r')
plt.scatter(pareto_front.area, pareto_front.delay, c='g')
plt.grid()